In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('shopee.csv')

In [ ]:
df.head(5)

In [ ]:
current_rate = 13.10
df['original_price'] = df['price_ori'] * current_rate 
df['final_price'] = df['price_actual'] * current_rate

In [ ]:
df.columns

In [ ]:
df[['category_1', 'category_2', 'category_3']] = df['item_category_detail'].str.replace("Shopee | ", "").str.split('|', expand=True)

In [ ]:
df['specification'] = df['specification'].str.replace("Product Specifications Category Shopee ", "")

In [ ]:
df['item_rating'] = pd.to_numeric(df['item_rating'], errors='coerce')

In [ ]:
df['count_rating'] = df['total_rating'].str.replace(r'(\d+(\.\d+)?)k$', lambda x: str(float(x.group(1)) * 1000), regex=True).astype(float)

df['total_sold'] = df['total_sold'].str.replace(r'(\d+(\.\d+)?)k$', lambda x: str(float(x.group(1)) * 1000), regex=True).astype(float)

df['favorite'] = df['favorite'].str.replace('Favorite (',"")
df['favorite'] = df['favorite'].replace('label_favorite', 'N/A')
df['favorite'] = df['favorite'].replace('N/A', np.nan)
df['favorite'] = df['favorite'].str.replace(r'(\d+(\.\d+)?)k$', lambda x: str(float(x.group(1)) * 1000), regex=True).astype(float)

In [ ]:
df = df.drop(['delivery', 'sitename', 'pict_link', 'link_ori', 'desc', 'price_ori', 'price_actual','item_category_detail', 'total_rating', 'w_date'], axis=1)

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

In [ ]:
df = df.fillna(0)

In [ ]:
df['product_origin'] = df['specification'].str.extract(r'From\s+(.*)$')
df['specification'] = df['specification'].str.replace(r'\s+From\s+.*$', '', regex=True)

In [ ]:
df['product_origin'] = df['product_origin'].apply(lambda x: str(x).split()[-1] if pd.notna(x) else x)

In [ ]:
mapping = {
    "Sembilan": "Negeri Sembilan",
    "Pinang": "Pulau Pinang",
    "Lumpur": "Kuala Lumpur",
    "ID": "Indonesia"
}

df['product_origin'] = df['product_origin'].replace(mapping)

In [ ]:
# for Market Basket Analysis CSV
df['category_hierarchy'] = df[['category_1', 'category_2', 'category_3']]
    .apply(lambda x: ' > '.join(x.dropna()), axis=1)
df = df.drop(['category_1', 'category_2','category_3'], axis=1)

df.to_csv('shopee-category-hierarchy.csv')

In [ ]:
# for predictive model (in case)
df[['category_1', 'category_2', 'category_3']] = df['category_hierarchy'].str.split('>', expand=True)
df.drop(['category_hierarchy'], axis=1)

df.to_csv('shopee-category-separated.csv')